<a href="https://colab.research.google.com/github/HyunmookKim/zen-koan-nlp-analysis/blob/master/src/preprocessing/01_text_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# ============================================
# 선 공안 NLP 연구 - 텍스트 추출
# ============================================

!pip install pymupdf -q

from google.colab import drive
drive.mount('/content/drive')

import fitz
import re
import os
import json
from datetime import datetime
from collections import Counter

BASE_DIR = "/content/drive/MyDrive/raw"
OUTPUT_DIR = "/content/drive/MyDrive/processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def remove_control_chars(text):
    return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', text)

def remove_all_chinese(text):
    patterns = [r'[\u4e00-\u9fff]', r'[\u3400-\u4dbf]', r'[\u2e80-\u2eff]', r'[\u2f00-\u2fdf]', r'[\u3000-\u303f]', r'[\uff00-\uffef]', r'[\u31c0-\u31ef]', r'[\uf900-\ufaff]']
    for p in patterns:
        text = re.sub(p, '', text)
    return text

def remove_page_numbers(text):
    lines = text.split('\n')
    cleaned = []
    for line in lines:
        stripped = line.strip()
        if re.match(r'^[0-9]+[a-c]?$', stripped):
            continue
        if re.match(r'^[ivxlcdm]+$', stripped.lower()) and len(stripped) < 10:
            continue
        cleaned.append(line)
    return '\n'.join(cleaned)

def remove_running_headers(text, headers):
    lines = text.split('\n')
    cleaned = []
    for line in lines:
        stripped = line.strip()
        skip = False
        for h in headers:
            if stripped == h or stripped.lower() == h.lower():
                skip = True
                break
        if not skip:
            cleaned.append(line)
    return '\n'.join(cleaned)

def auto_detect_running_headers(text, threshold=10):
    lines = text.split('\n')
    line_counts = Counter(line.strip() for line in lines if line.strip())
    headers = []
    for line, count in line_counts.items():
        if count >= threshold and 3 < len(line) < 60:
            if not re.match(r'^[0-9]+$', line):
                headers.append(line)
    return headers

def remove_ctext_artifacts(text):
    """Chinese Text Project 웹사이트 문구 제거"""
    # 줄 단위 제거
    lines = text.split('\n')
    cleaned = []
    skip_patterns = [
        r'^\[Frequencies\]',
        r'^\[Text tools\]',
        r'^Library$',
        r'^Resources$',
        r'^English translation:',
        r'^James\s*$',
        r'^:$',
        r'^\[.*\]$',
    ]
    for line in lines:
        stripped = line.strip()
        skip = False
        for p in skip_patterns:
            if re.match(p, stripped):
                skip = True
                break
        if not skip:
            cleaned.append(line)
    text = '\n'.join(cleaned)

    # 추가: "James Legge [?]" 및 "Legge [?]" 제거
    text = re.sub(r'James\s+Legge\s*\[\?\]', '', text)
    text = re.sub(r'Legge\s*\[\?\]', '', text)

    return text

def clean_text(text):
    if not text:
        return ""
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    text = re.sub(r'[^\S\n]+', ' ', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    lines = [line.strip() for line in text.split('\n')]
    return '\n'.join(lines).strip()

def full_clean(text, remove_chinese=False, remove_ctext=False, extra_headers=None):
    text = remove_control_chars(text)
    if remove_chinese:
        text = remove_all_chinese(text)
    if remove_ctext:
        text = remove_ctext_artifacts(text)
    text = remove_page_numbers(text)
    auto_headers = auto_detect_running_headers(text)
    all_headers = auto_headers + (extra_headers or [])
    if all_headers:
        text = remove_running_headers(text, all_headers)
    return clean_text(text)

def extract_pdf_full(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text() + "\n"
    doc.close()
    return text

def extract_pdf_pages(pdf_path, start_page, end_page=None):
    doc = fitz.open(pdf_path)
    end = end_page if end_page else len(doc)
    text = ""
    for i in range(start_page, min(end, len(doc))):
        text += doc[i].get_text() + "\n"
    doc.close()
    return text

def find_pdf(folder, keyword):
    folder_path = os.path.join(BASE_DIR, folder)
    if not os.path.exists(folder_path):
        return None
    for f in os.listdir(folder_path):
        if keyword.lower() in f.lower() and f.endswith('.pdf'):
            return os.path.join(folder_path, f)
    return None

def save_result(text, filename, source_info):
    if not text or len(text.strip()) < 100:
        print(f"  ❌ {filename}: 내용 없음")
        return None
    path = os.path.join(OUTPUT_DIR, filename)
    with open(path, 'w', encoding='utf-8') as f:
        f.write(text)
    words = len(text.split())
    print(f"  ✅ {filename}: {words:,} words")
    return {"file": filename, "words": words, "source": source_info}

# ============================================
# 1. 공안
# ============================================

def extract_mumonkan():
    print("\n📖 [1/15] 무문관")
    pdf_path = find_pdf("koans", "mumonkan")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    text = remove_control_chars(text)
    text = text.replace('\u2019', "'").replace('\u2018', "'")
    pattern = r"THE CASE\s*\n(.*?)(?=WU-MEN'S COMMENT)"
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    result = ""
    for i, case in enumerate(matches, 1):
        c = case
        c = re.sub(r'PLATE \d+:.*?(?=\n)', '', c)
        c = re.sub(r'Courtesy of.*?(?=\n)', '', c)
        c = re.sub(r'Trans\. by.*?(?=\n)', '', c)
        c = clean_text(c)
        if c and len(c.strip()) > 20:
            result += f"\n=== CASE {i} ===\n{c.strip()}\n"
    return save_result(full_clean(result), "koans_mumonkan.txt", "Mumonkan, Robert Aitken")

def extract_blue_cliff():
    print("\n📖 [2/15] 벽암록")
    pdf_path = find_pdf("koans", "blue cliff")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    text = remove_control_chars(text)
    pattern = r'\nCASE\s*\n(.*?)(?=\nNOTES\s*\n)'
    matches = re.findall(pattern, text, re.DOTALL)
    result = ""
    for i, case in enumerate(matches, 1):
        c = full_clean(case, extra_headers=['THE BLUE CLIFF RECORD'])
        if c and len(c.strip()) > 30:
            result += f"\n=== CASE {i} ===\n{c.strip()}\n"
    return save_result(clean_text(result), "koans_blue_cliff.txt", "Blue Cliff Record, Thomas Cleary")

# ============================================
# 2. 불교 경전
# ============================================

def extract_diamond():
    print("\n📖 [3/15] 금강경")
    pdf_path = find_pdf("sutras", "diamond")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    m = re.search(r'Chapter\s*1\.?\s', text, re.IGNORECASE)
    if m:
        text = text[m.start():]
    m = re.search(r'(bowed to him and departed|prostrated themselves before the Buddha)', text, re.IGNORECASE)
    if m:
        text = text[:m.end()]
    return save_result(full_clean(text, extra_headers=['THE DIAMOND SUTRA']), "sutras_diamond.txt", "Diamond Sutra")

def extract_heart():
    print("\n📖 [4/15] 반야심경")
    pdf_path = find_pdf("sutras", "heart")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    for p in [r'The Heart Sutra in Sanskrit', r'pronunciation guide', r'SANSKRIT']:
        m = re.search(p, text, re.IGNORECASE)
        if m:
            text = text[:m.start()]
            break
    return save_result(full_clean(text), "sutras_heart.txt", "Heart Sutra")

def extract_platform():
    print("\n📖 [5/15] 육조단경")
    pdf_path = find_pdf("sutras", "platform")
    if not pdf_path:
        return None
    text = extract_pdf_pages(pdf_path, 34)
    m = re.search(r'End of The Platform Sutra.*?Sixth Patriarch\.?', text, re.IGNORECASE | re.DOTALL)
    if m:
        text = text[:m.end()]
    text = re.sub(r'\n36[0-9][a-c]\n', '\n', text)
    return save_result(full_clean(text, extra_headers=['The Platform Sutra of the Sixth Patriarch', 'Platform Sutra of the Dharma Treasure']), "sutras_platform.txt", "Platform Sutra, BDK")

def extract_lotus():
    print("\n📖 [6/15] 법화경")
    pdf_path = find_pdf("sutras", "lotus")
    if not pdf_path:
        return None
    text = extract_pdf_pages(pdf_path, 18)
    m = re.search(r'bowed to him and departed\.?', text, re.IGNORECASE)
    if m:
        text = text[:m.end()]
    return save_result(full_clean(text), "sutras_lotus.txt", "Lotus Sutra, BDK")

def extract_lankavatara():
    print("\n📖 [7/15] 능가경")
    pdf_path = find_pdf("sutras", "lankavatara")
    if not pdf_path:
        return None
    text = extract_pdf_pages(pdf_path, 35)
    m = re.search(r'This marks the end of the Lankavatara Sutra\.?', text, re.IGNORECASE)
    if m:
        text = text[:m.end()]
    return save_result(full_clean(text), "sutras_lankavatara.txt", "Lankavatara Sutra, Red Pine")

def extract_vimalakirti():
    print("\n📖 [8/15] 유마경")
    pdf_path = find_pdf("sutras", "vimalakirti")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    m = re.search(r'Thus have I heard', text, re.IGNORECASE)
    if m:
        text = text[m.start():]
    return save_result(full_clean(text), "sutras_vimalakirti.txt", "Vimalakirti Sutra, Robert Thurman")

def extract_avatamsaka():
    print("\n📖 [9/15] 화엄경")
    pdf_path = find_pdf("sutras", "avatamsaka")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    m = re.search(r'Chapter One[:\s]|CHAPTER ONE', text, re.IGNORECASE)
    if m:
        text = text[m.start():]
    m = re.search(r'\nEndnotes\s*\n', text, re.IGNORECASE)
    if m:
        text = text[:m.start()]
    return save_result(full_clean(text, extra_headers=['The Flower Adornment Sutra']), "sutras_avatamsaka.txt", "Avatamsaka Sutra, BDK")

def extract_amitabha():
    print("\n📖 [10/15] 아미타경")
    pdf_path = find_pdf("sutras", "amitabha")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    m = re.search(r'Thus I have heard', text, re.IGNORECASE)
    if m:
        text = text[m.start():]
    return save_result(full_clean(text), "sutras_amitabha.txt", "Amitabha Sutra")

# ============================================
# 3. 유교 경전
# ============================================

def extract_analects():
    print("\n📖 [11/15] 논어")
    pdf_path = find_pdf("confucian", "analects")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    return save_result(full_clean(text, remove_chinese=True, remove_ctext=True), "confucian_analects.txt", "Analects, James Legge")

def extract_mencius():
    print("\n📖 [12/15] 맹자")
    pdf_path = find_pdf("confucian", "mencius")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    return save_result(full_clean(text, remove_chinese=True, remove_ctext=True), "confucian_mencius.txt", "Mencius, James Legge")

def extract_doctrine():
    print("\n📖 [13/15] 중용·대학")
    pdf_path = find_pdf("confucian", "doctrine")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    m = re.search(r'I\.?\s*TEXT', text)
    if m:
        text = text[m.start():]
    m = re.search(r'That is the ultimate!', text, re.IGNORECASE)
    if m:
        text = text[:m.end()]
    return save_result(full_clean(text, remove_chinese=True, extra_headers=['The Great Learning', 'The Doctrine of the Mean']), "confucian_doctrine_mean.txt", "Doctrine of Mean + Great Learning")

def extract_filial():
    print("\n📖 [14/15] 효경")
    pdf_path = find_pdf("confucian", "filial")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    m = re.search(r'disaster is sure to follow\.', text, re.IGNORECASE)
    if m:
        text = text[:m.end()]
    text = re.sub(r'Primary Source Document.*?DBQ[s]?\)?', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'Asia for Educators.*?reserved\.?', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'http://afe\.easia\.columbia\.edu', '', text)
    text = re.sub(r'Page \d+ of \d+', '', text)
    text = re.sub(r'Columbia University', '', text)
    return save_result(full_clean(text, remove_chinese=True), "confucian_filial_piety.txt", "Classic of Filiality")

# ============================================
# 4. 일반 산문
# ============================================

def extract_prose():
    print("\n📖 [15/15] 당송 산문")
    pdf_path = find_pdf("prose", "gems")
    if not pdf_path:
        return None
    text = extract_pdf_full(pdf_path)
    m = re.search(r'THE DUKE OF CHOU', text, re.IGNORECASE)
    if m:
        text = text[m.start():]

    # 마지막 "THE END." 찾기 (INDEX 바로 앞)
    matches = list(re.finditer(r'THE\s+END\.', text, re.IGNORECASE))
    if matches:
        last_end = matches[-1]  # 마지막 매칭
        text = text[:last_end.end()]

    return save_result(full_clean(text), "prose_gems_chinese.txt", "Gems of Chinese Literature, Herbert Giles 1923")

# ============================================
# 메인 실행
# ============================================

print("=" * 60)
print("📚 PDF 텍스트 추출 v7.0")
print("=" * 60)

results = []
results.append(extract_mumonkan())
results.append(extract_blue_cliff())
results.append(extract_diamond())
results.append(extract_heart())
results.append(extract_platform())
results.append(extract_lotus())
results.append(extract_lankavatara())
results.append(extract_vimalakirti())
results.append(extract_avatamsaka())
results.append(extract_amitabha())
results.append(extract_analects())
results.append(extract_mencius())
results.append(extract_doctrine())
results.append(extract_filial())
results.append(extract_prose())

print("\n" + "=" * 60)
success = [r for r in results if r]
total_words = sum(r['words'] for r in success)
print(f"✅ 성공: {len(success)}개 / 총 {total_words:,} words")

with open(os.path.join(OUTPUT_DIR, "extraction_log.json"), 'w', encoding='utf-8') as f:
    json.dump({"version": "7.0", "date": datetime.now().isoformat(), "results": success, "total_words": total_words}, f, ensure_ascii=False, indent=2)

print(f"\n완료: {OUTPUT_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📚 PDF 텍스트 추출 v7.0

📖 [1/15] 무문관
  ✅ koans_mumonkan.txt: 4,797 words

📖 [2/15] 벽암록
  ✅ koans_blue_cliff.txt: 7,579 words

📖 [3/15] 금강경
  ✅ sutras_diamond.txt: 7,724 words

📖 [4/15] 반야심경
  ✅ sutras_heart.txt: 264 words

📖 [5/15] 육조단경
  ✅ sutras_platform.txt: 29,212 words

📖 [6/15] 법화경
  ✅ sutras_lotus.txt: 81,524 words

📖 [7/15] 능가경
  ✅ sutras_lankavatara.txt: 90,272 words

📖 [8/15] 유마경
  ✅ sutras_vimalakirti.txt: 36,425 words

📖 [9/15] 화엄경
  ✅ sutras_avatamsaka.txt: 258,501 words

📖 [10/15] 아미타경
  ✅ sutras_amitabha.txt: 1,829 words

📖 [11/15] 논어
  ✅ confucian_analects.txt: 28,619 words

📖 [12/15] 맹자
  ✅ confucian_mencius.txt: 60,474 words

📖 [13/15] 중용·대학
  ✅ confucian_doctrine_mean.txt: 12,560 words

📖 [14/15] 효경
  ✅ confucian_filial_piety.txt: 859 words

📖 [15/15] 당송 산문
  ✅ prose_gems_chinese.txt: 98,578 words

✅ 성공: 15개 / 총 719,217 words

완료: /content/drive